In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd
import pandas.util.testing as testing

import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

/var/folders/z4/hdvs0wyx3hsgmt7j31w_d2w40000gn/T/ipykernel_17278/3034166604.py:4: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as testing


In [3]:
def extract_year(df):
    """
    Extract year from date column of the dataframe.

    Args:
        df (pd.DataFrame): DataFrame for adding the column

    Returns:
        pd.DataFrame: DataFrame with "year" column
    """
    df["year"] = df.date.dt.year
    return df

In [5]:
!mkdir tmp/

In [6]:
path = "tmp/"

In [7]:
df1 = pd.DataFrame({"field1": ["a", "b", "c"]})
df1['key'] = 1
df1.key = df1.key.astype("int64")

df2 = pd.DataFrame({"field1": ["d", "e", "f"], "field2": ["j", "k", "l"]})
df2['key'] = 2
df2.key = df2.key.astype("int64")

# `quirk1` - consistent filenames

In [8]:
!rm -rf tmp/quirk1

In [9]:
consistent_path = f"{path}/quirk1/"

In [10]:
df1.to_parquet(
    consistent_path,
    engine="pyarrow",
    partition_cols="key",
    basename_template="file-{i}.parquet",
)

### workaround

In [22]:
df1.to_parquet(
    consistent_path, 
    engine="pyarrow",
    partition_cols="key",
    basename_template="file-{i}.parquet",
    existing_data_behavior="overwrite_or_ignore",
)

# `quirk2` - merging schemas

## `quirk2.1` - simple merge

In [11]:
!rm -rf tmp/quirk2.1

In [12]:
merge_path_1 = f"{path}/quirk2.1/"

In [13]:
df1.to_parquet(merge_path_1, engine="pyarrow", partition_cols="key")
df2.to_parquet(merge_path_1, engine="pyarrow", partition_cols="key")

In [14]:
pd.read_parquet(merge_path_1)

,field1,key
0,a,1
1,b,1
2,c,1
3,d,2
4,e,2
5,f,2


### workaround

In [15]:
df1_schema = pa.Schema.from_pandas(df1)
df2_schema = pa.Schema.from_pandas(df2)

schema = pa.unify_schemas((df1_schema, df2_schema))

pd.read_parquet(merge_path_1, schema=schema)

,field1,key,field2
0,a,1,None
1,b,1,None
2,c,1,None
3,d,2,j
4,e,2,k
5,f,2,l


## `quirk2.2` - inconsistent dtypes

In [16]:
!rm -rf tmp/quirk2.2

In [17]:
df3 = pd.DataFrame({"field1": [1, 2, 3], "field2": ["j", "k", "l"]})
df3['key'] = 2
df3.key = df3.key.astype("int64")

In [18]:
merge_path_2 = f"{path}/quirk2.2/"

In [19]:
df1_schema = pa.Schema.from_pandas(df1)
df3_schema = pa.Schema.from_pandas(df3)

schema = pa.unify_schemas((df1_schema, df3_schema))

ArrowInvalid: Unable to merge: Field field1 has incompatible types: string vs int64

### workaround

In [20]:
df1 = df1.astype("string[pyarrow]")
df3 = df3.astype("string[pyarrow]")

In [21]:
df1_schema = pa.Schema.from_pandas(df1)
df3_schema = pa.Schema.from_pandas(df3)

schema = pa.unify_schemas((df1_schema, df3_schema))

In [22]:
df1.to_parquet(merge_path_2, engine="pyarrow", partition_cols="key")
df3.to_parquet(merge_path_2, engine="pyarrow", partition_cols="key")

In [23]:
pd.read_parquet(merge_path_2, schema=schema)

,field1,key,field2
0,a,1,None
1,b,1,None
2,c,1,None
3,1,2,j
4,2,2,k
5,3,2,l


# `quirk3` - saving schema

In [24]:
!rm -rf tmp/quirk3

In [25]:
saving_path = f"{path}/quirk3/"

In [26]:
np.random.seed(42)

df = (
    testing.makeTimeDataFrame(nper=10_000)
    .reset_index()
    .rename(columns={"index": "date"})
    .pipe(extract_year)
)

print(df.shape)
df.head()

(10000, 6)


,date,A,B,C,D,year
0,2000-01-03,0.496714,-0.678495,0.348286,-1.980572,2000
1,2000-01-04,-0.138264,-0.305499,0.283324,-1.054986,2000
2,2000-01-05,0.647689,-0.597381,-0.936520,-0.587028,2000
3,2000-01-06,1.523030,0.110418,0.579584,0.149669,2000
4,2000-01-07,-0.234153,1.197179,-1.490083,1.024162,2000


In [27]:
df.to_parquet(
        saving_path,
        engine="pyarrow",
        partition_cols="year")

In [28]:
schema = pa.Schema.from_pandas(df)
schema

date: timestamp[ns]
A: double
B: double
C: double
D: double
year: int64
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 896

In [29]:
pq.write_metadata(schema, f"{saving_path}/_common_metadata")

In [30]:
# timestamp changed from ns to us in `date` column
pq.read_schema(f"{saving_path}/_common_metadata")

date: timestamp[us]
A: double
B: double
C: double
D: double
year: int64
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 896

### workaround

In [31]:
pq.write_metadata(schema, f"{saving_path}/_common_metadata", use_deprecated_int96_timestamps=True)
pq.read_schema(f"{saving_path}/_common_metadata")

date: timestamp[ns]
A: double
B: double
C: double
D: double
year: int64
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 896

# quirk4 - single value in partition col

In [32]:
!rm -rf tmp/quirk4

In [33]:
single_partition_path = f"{path}/quirk4/"

In [34]:
y2038 = df[df["year"] == 2038]

In [35]:
y2038.to_parquet(
    single_partition_path,
    engine="pyarrow",
    partition_cols="year"
)

In [36]:
read_y2038 = pd.read_parquet(single_partition_path, engine="pyarrow")
read_y2038.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 9914 to 9999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    86 non-null     datetime64[ns]
 1   A       86 non-null     float64       
 2   B       86 non-null     float64       
 3   C       86 non-null     float64       
 4   D       86 non-null     float64       
 5   year    86 non-null     category      
dtypes: category(1), datetime64[ns](1), float64(4)
memory usage: 4.2 KB


### workaround

In [37]:
schema = pa.Schema.from_pandas(df)
pq.write_metadata(schema, f"{single_partition_path}/_common_metadata", use_deprecated_int96_timestamps=True)

In [38]:
read_schema = pq.read_schema(f"{single_partition_path}/_common_metadata")
read_schema

date: timestamp[ns]
A: double
B: double
C: double
D: double
year: int64
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 896

In [39]:
read_y2038 = pd.read_parquet(single_partition_path, engine="pyarrow", schema=read_schema)
read_y2038.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    86 non-null     datetime64[ns]
 1   A       86 non-null     float64       
 2   B       86 non-null     float64       
 3   C       86 non-null     float64       
 4   D       86 non-null     float64       
 5   year    86 non-null     int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 4.2 KB
